In [25]:
# !pip install openai

In [ ]:
import openai
OPENAI_API_KEY = 'API_KEY'

In [73]:
GPT_3_5_TURBO = "gpt-3.5-turbo"
GPT_4_TURBO_PREVIEW = "gpt-4-turbo-preview"
GPT_4 = 'gpt-4'

In [78]:
# Assuming OPENAI_API_KEY is defined elsewhere or use the actual API key string
client = openai.OpenAI(api_key=OPENAI_API_KEY)  # Replace "YOUR_API_KEY" with your actual OpenAI API key

def generate_finnish_sentences(words, model = GPT_3_5_TURBO):
    word_to_sentence = {}
    for word in words:
#         prompt = f"Generate a simple, useful, beginner-friendly sentence in Finnish using the word or phrase: '{word}'. The sentence should be useful and easy to understand for someone learning Finnish."
        prompt = f"Generate a simple, useful, beginner-friendly sentence in Finnish using the word or phrase: '{word}'. The sentence should be useful and easy to understand for someone learning Finnish. Provide only the Finnish sentence, without any translation or additional information."
        
        response = client.chat.completions.create(
            model=model,
            messages=[{
                "role": "user",
                "content": prompt
            }]
        )
        
#         print(response.choices[0].message)        
        
        # Correctly accessing the response data
        # Ensure you're correctly interpreting the structure of the response
        sentence = response.choices[0].message.content if response.choices else 'No response generated.'
        
        # Map the word or phrase to the generated sentence
        word_to_sentence[word] = sentence.strip()
    return word_to_sentence

# # List of Finnish words or phrases you want to create sentences for
# finnish_words = ['koira', 'on kaunis päivä', 'rakastan oppia suomea']

# # Generate sentences and store them in a dictionary
# word_to_sentence_map = generate_finnish_sentences(finnish_words)

# # Print the word-to-sentence hash-map
# for word, sentence in word_to_sentence_map.items():
#     print(f"'{word}': '{sentence}'")


In [70]:
import openai
from tqdm.notebook import tqdm

system_prompt = "You are a professional russian-finnish translator. Use 'ё' in russian written language instead of 'е', when it is grammatically more correct." # When 'е' suits better use it." #" When 'е' suits better use it."

def translate_word_or_phrase_to_russian(phrase, model = GPT_3_5_TURBO):
    prompt = f"Translate the following Finnish phrase to Russian: '{phrase}'. Provide only the translation to russian sentence, without any additional information."
    response = client.chat.completions.create(
            model = model,
            messages = [{
                "role": "system",
                "content": system_prompt
            },
                {
                "role": "user",
                "content": prompt
            }],
            temperature = 0.0
        )
        
    # Correctly accessing the response data
    # Ensure you're correctly interpreting the structure of the response
    translation = response.choices[0].message.content if response.choices else 'No response generated.'

    translation = translation.strip()
    return translation

def translate_word_or_phrase_to_russian_with_context(phrase, context, model = GPT_3_5_TURBO):
    prompt = f"""Translate the following Finnish word to Russian: ###'{phrase}'###. \n 
                To understand tranlsation context for words with different variants of translation use the following sentence with the word: '{context}'. 
                Provide only the translation to russian word, without any additional information."""
    response = client.chat.completions.create(
            model = model,
            messages=[{
                "role": "system",
                "content": system_prompt
            },
                {
                "role": "user",
                "content": prompt
            }],
            temperature = 0.0
        )
        
    # Correctly accessing the response data
    # Ensure you're correctly interpreting the structure of the response
    translation = response.choices[0].message.content if response.choices else 'No response generated.'

    translation = translation.strip()
    return translation
    

def translate_to_russian(data, model = GPT_3_5_TURBO):
    translations = []
    for finnish_word, finnish_sentence in tqdm(data, desc="Translating to russian"):
        # Construct the prompt for translation
        if finnish_sentence:
            russian_sentence = translate_word_or_phrase_to_russian(finnish_sentence, model=model)
            russian_word = translate_word_or_phrase_to_russian_with_context(finnish_word, finnish_sentence, model=model)
            translations.append((finnish_word, finnish_sentence, russian_word, russian_sentence))
        else:
            russian_word = translate_word_or_phrase_to_russian(finnish_word, model=model)
            translations.append((finnish_word, finnish_sentence, russian_word, ""))
    return translations


In [55]:
# translate_to_russian([["juodaan", "Juodaan vettä ruoan kanssa."], ["Menemme kotiin.", ""]])
input_data = [['reissu', 'Menen huomenna kauppaan, haluaisitko tulla mukaan reissuun?'],
 ['olla tehtävänä', 'Minulla on tehtävänä siivota huoneeni tänään.'],
 ['viime', 'Viime viikolla kävin elokuvissa.'],
 ["juodaan", "Juodaan vettä ruoan kanssa."], ["Menemme kotiin.", ""],
             ['hostelli',
  'Minä yövyn hostellissa lomani aikana.']
]
translate_to_russian(input_data)

Translating to russian:   0%|          | 0/6 [00:00<?, ?it/s]

[('reissu',
  'Menen huomenna kauppaan, haluaisitko tulla mukaan reissuun?',
  'поездка',
  'Завтра я иду в магазин, хочешь пойти со мной?'),
 ('olla tehtävänä',
  'Minulla on tehtävänä siivota huoneeni tänään.',
  'иметь задание',
  'У меня есть задание убрать свою комнату сегодня.'),
 ('viime',
  'Viime viikolla kävin elokuvissa.',
  'прошлый',
  'На прошлой неделе я ходил в кино.'),
 ('juodaan', 'Juodaan vettä ruoan kanssa.', 'пьём', 'Пьём воду с едой.'),
 ('Menemme kotiin.', '', 'Мы идём домой.', ''),
 ('hostelli',
  'Minä yövyn hostellissa lomani aikana.',
  'хостёл',
  'Я ночую в хостеле во время моего отпуска.')]

In [ ]:
translate_to_russian([["juodaan", "Juodaan vettä ruoan kanssa."], ["Menemme kotiin.", ""]])

## Check proper russian

In [62]:
import openai

openai.api_key = 'your_openai_api_key'

def check_and_fix_russian(sentence):
    CORRECT_CODE = '#ok#'
    prompt = f"Check if the following Russian sentence is grammatically correct and idiomatic. Double check for correct usage of 'ё'. If it is, reply with '{CORRECT_CODE}'. If it is not, provide a corrected version:\n\n'{sentence}'"
    
    response = client.chat.completions.create(
#             model="gpt-3.5-turbo",
#             model="gpt-3.5-turbo",
            model='gpt-4-turbo-preview',
            messages=[{
                "role": "user",
                "content": prompt
            }],
            temperature = 0.0
        )
        
    # Correctly accessing the response data
    # Ensure you're correctly interpreting the structure of the response
    result = response.choices[0].message.content if response.choices else '#noresponse#'
    result = result.strip()
    if result == CORRECT_CODE:
        return True, ""
    else:
        return False, result
    return translation


In [63]:
# Example usage
russian_sentence = "Твой примёр предложения на русском."
is_ok, fixed_sentence = check_and_fix_russian(russian_sentence)
print(is_ok)
print("Result:", fixed_sentence)

False
Result: 'Твой пример предложения на русском.'


In [65]:
sentences2check = ["Мы уютно дома сегодня.",
"хостёл",
"сразу",
"Пью в мыслях свой кофе.",
"Можем ли мы договориться о времени на завтрашний день?",
"Я положил ключи везде.	",
"Использую интернёт ежедневно.",
"В любом случаё.",
"В любом случаё нужна помощь."
]
for s in sentences2check:
    is_ok, fixed_sentence = check_and_fix_russian(s)
    print("=======")
    print(s)
    print(is_ok)
    print("Result:", fixed_sentence)

Мы уютно дома сегодня.
False
Result: Мы уютно дома сегодня.

Corrected version:
Мы сегодня уютно дома.
хостёл
True
Result: 
сразу
True
Result: 
Пью в мыслях свой кофе.
True
Result: 
Можем ли мы договориться о времени на завтрашний день?
True
Result: 
Я положил ключи везде.	
False
Result: 'Я положил ключи везде.' - This sentence is grammatically correct, but it might not convey the intended meaning effectively. The sentence literally means "I put the keys everywhere," which is unusual or nonsensical in most contexts. If the intended meaning is "I put the keys somewhere, but I don't remember where," a more idiomatic way to express this in Russian would be:

'Я положил ключи где-то.'

However, if the original sentence precisely conveys the intended meaning, then it is grammatically correct as is.
Использую интернёт ежедневно.
False
Result: 'Использую интернет ежедневно.'
В любом случаё.
False
Result: 'В любом случае.'
В любом случаё нужна помощь.
False
Result: 'В любом случае нужна помощь

In [68]:
# # Translate the data to Russian
# translated_data = gptutils.translate_to_russian(input_data)

input_data = [["reissu", "Menen huomenna kauppaan, haluaisitko tulla mukaan reissuun?"],
["mukavasti", "Olemme mukavasti kotona tänään."],
["hostelli", "Minä yövyn hostellissa lomani aikana."],
["heti", "Menen kauppaan heti."],
["mielellään", "Juon miellessäni kahvia."],
["sopia", "Voimmeko sopia ajan huomiselle?"],
["joka paikkaan", "Minä laitoin avaimet joka paikkaan."],
["netti (internet)", "Käytän nettiä päivittäin."],
["joka tapauksessa", "Joka tapauksessa tarvitsen apua."]]

input_data2 


In [71]:
translated_data = translate_to_russian(input_data, model = GPT_4_TURBO_PREVIEW)

Translating to russian:   0%|          | 0/9 [00:00<?, ?it/s]

In [72]:
translated_data

[('reissu',
  'Menen huomenna kauppaan, haluaisitko tulla mukaan reissuun?',
  'поездка',
  'Я иду завтра в магазин, хотел бы ты присоединиться ко мне в поездке?'),
 ('mukavasti',
  'Olemme mukavasti kotona tänään.',
  'удобно',
  'Мы сегодня уютно дома.'),
 ('hostelli',
  'Minä yövyn hostellissa lomani aikana.',
  'хостеле',
  'Я останавливаюсь в хостеле во время своего отпуска.'),
 ('heti', 'Menen kauppaan heti.', 'сразу', 'Иду в магазин сразу.'),
 ('mielellään',
  'Juon miellessäni kahvia.',
  'охотно',
  'Я пью кофе в своих мыслях.'),
 ('sopia',
  'Voimmeko sopia ajan huomiselle?',
  'договориться',
  'Можем ли мы договориться о времени на завтра?'),
 ('joka paikkaan',
  'Minä laitoin avaimet joka paikkaan.',
  'везде',
  'Я положил ключи вездё.'),
 ('netti (internet)',
  'Käytän nettiä päivittäin.',
  'интернет',
  'Я использую интернет каждый день.'),
 ('joka tapauksessa',
  'Joka tapauksessa tarvitsen apua.',
  'В любом случае',
  'В любом случае мне нужна помощь.')]

In [74]:
translated_data2 = translate_to_russian(input_data, model = GPT_4)

Translating to russian:   0%|          | 0/9 [00:00<?, ?it/s]

In [75]:
translated_data2

[('reissu',
  'Menen huomenna kauppaan, haluaisitko tulla mukaan reissuun?',
  'поездка',
  'Я иду завтра в магазин, хотел бы ты присоединиться ко мне в поездке?'),
 ('mukavasti',
  'Olemme mukavasti kotona tänään.',
  'удобно',
  'Мы уютно дома сегодня.'),
 ('hostelli',
  'Minä yövyn hostellissa lomani aikana.',
  'хостел',
  'Я останусь в хостеле во время своего отпуска.'),
 ('heti', 'Menen kauppaan heti.', 'сразу', 'Я иду в магазин сразу.'),
 ('mielellään',
  'Juon miellessäni kahvia.',
  'с удовольствием',
  'Я пью кофе в своём воображении.'),
 ('sopia',
  'Voimmeko sopia ajan huomiselle?',
  'договориться',
  'Можем ли мы договориться о встрече на завтра?'),
 ('joka paikkaan',
  'Minä laitoin avaimet joka paikkaan.',
  'в каждое место',
  'Я положил ключи везде.'),
 ('netti (internet)',
  'Käytän nettiä päivittäin.',
  'интернет',
  'Я использую интернет каждый день.'),
 ('joka tapauksessa',
  'Joka tapauksessa tarvitsen apua.',
  'в любом случае',
  'В любом случае, мне нужна пом

In [77]:
# sentence examples
words = ["reissu",
"mukavasti",
"hostelli",
"heti",
"mielellään",
"sopia",
"joka paikkaan",
"netti (internet)",
"joka tapauksessa"]

In [79]:
res  = generate_finnish_sentences(words, model = GPT_4)

In [80]:
res

{'reissu': 'Hän lähti pitkälle reissulle maailman ympäri.',
 'mukavasti': 'Olemme mukavasti kotona katsomassa elokuvaa.',
 'hostelli': 'Minä varasin huoneen hostellista viikonlopuksi.',
 'heti': 'Tarvitsemme apua heti.',
 'mielellään': 'Hän ottaisi mielellään lasin vettä.',
 'sopia': 'Kun sopimme tapaamisen ajan, lähetän sinulle kalenterikutsun.',
 'joka paikkaan': 'Minulla on tapana ottaa vesipullo mukaani joka paikkaan.',
 'netti (internet)': 'Netti ei toimi, voitko korjata sen?',
 'joka tapauksessa': 'Joka tapauksessa, meidän täytyy lähteä aikaisin aamulla.'}